In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, CheckboxButtonGroup, CustomJS, HoverTool
from bokeh.layouts import column
from bokeh.palettes import Spectral5
from bokeh.palettes import Colorblind5
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Category20, Category20b, Category20c
from math import pi
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data=pd.read_csv('electric-car-sales-share.csv')

In [ ]:
required_countries = ["United States", "European Union (27)", "China", "Denmark", "Norway"]
required_countries_data = data[data['Entity'].isin(required_countries)]

output_notebook()

#ColumnDataSource from the DataFrame
source = ColumnDataSource(required_countries_data)

#Bokeh figure
p = figure(title="Electric Car Sales Share Comparison",
           x_axis_label='Year', y_axis_label='Electric Cars Sales Share (%)')

#each country's data with a unique color
colors = Colorblind5  
renderers = {}
for i, country in enumerate(required_countries):
    country_data = required_countries_data[required_countries_data['Entity'] == country]
    cds = ColumnDataSource(country_data)
    renderers[country] = p.line(x='Year', y='Share of new cars that are electric', 
                                source=cds, color=colors[i % len(colors)], 
                                line_width=2, legend_label=country)

p.legend.click_policy = "hide"
p.legend.location = 'top_left'
#CheckboxButtonGroup with all countries selected by default
checkbox_button_group = CheckboxButtonGroup(labels=required_countries, 
                                            active=list(range(len(required_countries))))

#JavaScript callback to show/hide lines
callback = CustomJS(args=dict(renderers=renderers), code="""
    for (var country in renderers) {
        renderers[country].visible = false;
    }
    for (var i of cb_obj.active) {
        var country = cb_obj.labels[i];
        renderers[country].visible = true;
    }
""")
checkbox_button_group.js_on_change('active', callback)
layout = column(checkbox_button_group, p)
show(layout)

Loading BokehJS ...

: 

In [ ]:
sales_data = pd.read_csv('Electric Car Sales by Model in USA.csv')

output_notebook()

sales_by_model = sales_data.groupby(['Make', 'Model']).sum().reset_index()
#Pivot data to have years as columns, models as rows, and sales as values
model_yearly_sales = pd.melt(sales_by_model, id_vars=['Make', 'Model'], var_name='Year', value_name='Sales')
model_yearly_sales = model_yearly_sales[model_yearly_sales['Year'].isin([str(year) for year in range(2012, 2020)])]
#Make and Model into a single column for the legend
model_yearly_sales['Make-Model'] = model_yearly_sales['Make'] + " - " + model_yearly_sales['Model']

#colors for each model
unique_models = model_yearly_sales['Make-Model'].unique().tolist()
TolRainbow21 = ['#e6194B', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',
    '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',
    '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#000000',
    '#8F00FF', '#aa6e28'] * 3
colors = TolRainbow21[:len(unique_models)]
#color map from models to colors
model_to_color = {model: colors[i % len(colors)] for i, model in enumerate(unique_models)}
model_yearly_sales['Color'] = [model_to_color[model] for model in model_yearly_sales['Make-Model']]

#ColumnDataSource for Bokeh plot
source = ColumnDataSource(model_yearly_sales)

p = figure(x_range=model_yearly_sales['Year'].unique(), title="Total Car Sales by Model and Year",
           x_axis_label='Year', y_axis_label='Total Sales', sizing_mode="stretch_width", height=1400)

#Stack the bars for each model
for model in unique_models:
    model_data = model_yearly_sales[model_yearly_sales['Make-Model'] == model]
    source = ColumnDataSource(model_data)
    p.vbar(x='Year', top='Sales', width=0.8, source=source, color='Color', legend_label=model)

#legend
p.legend.click_policy = "hide"
p.legend.title = 'Make - Model'
# Add hover tool
hover = HoverTool()
hover.tooltips = [("Make - Model", "@Make- @Model"), ("Sales", "@Sales"), ("Year", "@Year")]
p.add_tools(hover)
p.xaxis.major_label_orientation = pi/4
p.add_layout(p.legend[0], 'right')
show(p)

Loading BokehJS ...